Firts execute the requirements with "sudo sh requirements"

In [43]:
!pip install hunspell
!pip install kagglehub
!pip install contractions
!pip install nltk
!pip install stanza

In [44]:
import stanza
from stanza.models.common.doc import Document
stanza.download("en")
nlp_stanza = stanza.Pipeline(lang="en", processors="tokenize")

2025-03-14 01:00:10 INFO: Downloaded file to /home/bramen/stanza_resources/resources.json
2025-03-14 01:00:10 INFO: Downloading default packages for language: en (English) ...
2025-03-14 01:00:11 INFO: File exists: /home/bramen/stanza_resources/en/default.zip
2025-03-14 01:00:14 INFO: Finished downloading models and saved to /home/bramen/stanza_resources
2025-03-14 01:00:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-03-14 01:00:14 INFO: Downloaded file to /home/bramen/stanza_resources/resources.json
2025-03-14 01:00:14 WARNING: Language en package default expects mwt, which has been added
2025-03-14 01:00:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2025-03-14 01:00:14 INFO: Using device: cpu
2025-03-14 01:00:14 INFO: Lo

In [45]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jef1056/anime-subtitles")

print("Path to dataset files:", path)

folder = path + "/Anime Datasets V4/Preprocessed/Preprocessed"
output_folder = "./animes_subs/"

Path to dataset files: /home/bramen/.cache/kagglehub/datasets/jef1056/anime-subtitles/versions/24


Change the json files into .txt ones.

Execute this code if the files aren't created (they normally already are)

In [46]:
import contractions

def expand_contractions(text):
    return contractions.fix(text)

In [47]:
import hunspell

# Cargar diccionario en inglés (debe instalarse en el sistema)
h = hunspell.HunSpell('/usr/share/hunspell/en_US.dic', '/usr/share/hunspell/en_US.aff')

def expand_corrections(text: str) -> str:
    """Corrige palabras mal escritas usando HunSpell."""
    return " ".join(
        (h.suggest(word)[0] if not h.spell(word) and h.suggest(word) else word)
        for word in text.split()
    )


In [48]:
import re

def normalization(content: str) -> str:
    """Expande contracciones y normaliza el texto eliminando caracteres no alfabéticos."""
    norm_content = contractions.fix(content)
    # norm_content = expand_corrections(content)
    # norm_content = " ".join(re.findall(r"[a-zA-Z]+", norm_content)).lower()
    return norm_content


In [49]:
def limpiar_texto(texto):
    """Elimina signos de puntuación y caracteres no alfabéticos."""
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)  # Mantiene solo letras y espacios
    return texto.lower().strip()  # Convierte a minúsculas y elimina espacios extra

In [50]:
def create_corpus(words: list, noise=True):
    with open("corpus.txt", "a", encoding="utf-8") as new_file:
        if noise:
            content = "\n".join(words).strip() + "\n"
        else:
            result = "\n".join(filter(None, [limpiar_texto(word) for word in words]))
            if result:  # Solo escribe si no está vacío
                content = result + "\n"
            else:
                return  # Evita escribir líneas vacías

        new_file.write(content)

In [51]:
def clean_corpus():
    # Leer el archivo y convertirlo en una lista
    with open("corpus.txt", "r", encoding="utf-8") as file:
        words = file.read().splitlines()  # Convierte cada línea en un elemento de la lista

    # Eliminar palabras repetidas con un set y volver a lista ordenada
    unique_words = sorted(set(words))

    # Guardar las palabras únicas en un nuevo archivo
    with open("corpus_unique.txt", "w", encoding="utf-8") as file:
        file.write("\n".join(unique_words))

    print("Proceso completado. Se guardaron", len(unique_words), "palabras únicas en 'corpus_unique.txt'.")


In [52]:
import json

def segmentar_phrases(conversations, file_name):
    with open(file_name, "w", encoding="utf-8") as new_file:
        content_list = []
        for line in conversations[0]:
            line = line[0]
            line_stanza: Document = nlp_stanza(normalization(line))

            phrases = [sentence.text for sentence in line_stanza.sentences]
            words = [word.text for sentence in line_stanza.sentences for word in sentence.words]

            create_corpus(words, False)

            contetn_json = {
                line : {
                    "phrases": phrases,
                    "words": words
                    }
            }
            content_list.append(contetn_json)
        json.dump(content_list, new_file, indent=4, ensure_ascii=False)


In [ ]:




clean_corpus()



1 / 3836 [Ayako-BakaWolf]_Rosario___Vampire_-_13_-_[H264][1280x720]_[7608B35D].json
2 / 3836 DBZ - 050 - Unexpected Problem.json
3 / 3836 [HorribleSubs] Sora yori mo Tooi Basho - 12 [720p]_track3_eng.json
4 / 3836 - AMG 11 - [Exiled-Destiny] _ Ah! A Demon has Come and is Creating Calamity_Track5.json
5 / 3836 [HorribleSubs] Sakurasou no Pet na Kanojo - 10 [720p].json
6 / 3836 [Coalgirls] Darker than Black - 08 (BD 1280x720 x264  FLAC).json
7 / 3836 [Coalgirls]_Darker_than_Black_-_Ryuusei_no_Gemini_07_(1280x720_Blu-Ray_FLAC)_[0CC6D95A].json
8 / 3836 [TMD_A-Destiny]_Bakuman_05_[720p][04D19245].json
9 / 3836 [HorribleSubs] Mob Psycho 100 - 06 [720p]_track3_eng.json
10 / 3836 [HorribleSubs] Yahari Ore no Seishun Love Come wa Machigatteiru Zoku - 10 [720p].json
11 / 3836 [Horriblesubs] Fairy Tail - 02 [720p].json
12 / 3836 [ANBU-Frostii]_Kimi_ni_Todoke_-_11_[30D9D71C].json
13 / 3836 [HorribleSubs] Himouto! Umaru-chan R - 12 [720p]_track3_eng.json
14 / 3836 [HorribleSubs] Haikyuu!! S3 - 02 [